In [1]:
from kafka import KafkaConsumer
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# For first window

In [2]:
consumer = KafkaConsumer(
    'ml-raw-dns',
    bootstrap_servers="localhost:9092",
    auto_offset_reset='earliest',
    enable_auto_commit=False
)
a=[]
empty_Arr=[]
i=0
results=[]

for m in consumer:
    a.append(m)
    if(i==1000):
        break
    i=i+1

## Datamaker


In [3]:
for i in range(1,1001):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)

## Feature extraction


In [4]:
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
X_train, X_test, y_train, y_test = train_test_split(train_labels,y ,
                                   random_state=104, 
                                   test_size=0.20, 
                                   shuffle=True)


Accuracy from Static model

In [5]:
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.829


Making a dynamic model

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

xgb_clf = xgb.XGBClassifier(learning_rate =0.01, n_estimators=5000, max_depth=17, min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xgb_clf.fit(X_train, y_train)
xgb_pred = xgb_clf.predict(X_test)  

In [7]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, xgb_pred))

Accuracy: 0.83


In [8]:
import pickle
# save the model to disk
filename = 'xgboost_model_dynamic_window1.sav'
pickle.dump(xgb_clf, open(filename, 'wb'))

# For second window

In [9]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>1100 and i <= 2100):
        a.append(m)
    if(i==2100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.811


In [10]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, xgb_pred))

Accuracy: 0.83


In [11]:
filename = 'xgboost_model_dynamic_window1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.809


# For third window

In [12]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>2100 and i <= 3100):
        a.append(m)
    if(i==3100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.797


In [13]:
filename = 'xgboost_model_dynamic_window1.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.798


# Fourth window

In [14]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>3100 and i <= 4100):
        a.append(m)
    if(i==4100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.813


# Fifth window

In [15]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>4100 and i <= 5100):
        a.append(m)
    if(i==5100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.825


# Sixth window

In [16]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>5100 and i <= 6100):
        a.append(m)
    if(i==6100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.818


# seventh window

In [17]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>6100 and i <= 7100):
        a.append(m)
    if(i==7100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.829


# 8th window

In [18]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>7100 and i <= 8100):
        a.append(m)
    if(i==8100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.83


# Ninth window

In [19]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>8100 and i <= 9100):
        a.append(m)
    if(i==9100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.832


# Tenth window

In [20]:
a=[]
i=0
empty_Arr=[]
df2.drop(df2.index, inplace=True)
for m in consumer:
    if(i>9100 and i <= 10100):
        a.append(m)
    if(i==10100):
        break
    i=i+1
empty_Arr=[]
for i in range(0,1000):
    demo=(a[i][6]).decode("utf-8")
    demo=demo[2:-3]
    demo=demo.split(",")
    empty_Arr.append(demo)
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
for i in range(0,len(empty_Arr)):
    df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
           ignore_index=True)
fh = FeatureHasher(n_features=3, input_type='string')
#for feature timestamp
hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
hashed1 = pd.DataFrame(hashed1.todense())
hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
hashed2 = pd.DataFrame(hashed2.todense())
hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
hashed3 = fh.transform(df2[['sld']].astype(str).values)
hashed3 = pd.DataFrame(hashed3.todense())
hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
y = df2["Target Attack"]
y=y.astype(float)
#choose best features for training
train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
       'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
       'sld_fh2']]
train_labels = train_labels.astype(float)
import pickle
# save the model to disk
filename = 'xgboost_model.sav'
# pickle.dump(xgb_clf, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(train_labels, y)
results.append(result)
print(result)

0.824


In [21]:
results

[0.829,
 0.811,
 0.809,
 0.797,
 0.798,
 0.813,
 0.825,
 0.818,
 0.829,
 0.83,
 0.832,
 0.824]

In [22]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression

# generate dataset
# X, y = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=1, cluster_std=3)
X= train_labels.to_numpy()
X = X.iloc[:, :2].values
y=y.to_numpy()
# define bounds of the domain
min1, max1 = X[:, 0].min()-1, X[:, 0].max()+1
min2, max2 = X[:, 1].min()-1, X[:, 1].max()+1
# define the x and y scale
x1grid = np.arange(min1, max1, 0.1)
x2grid = np.arange(min2, max2, 0.1)
# create all of the lines and rows of the grid
xx, yy = np.meshgrid(x1grid, x2grid)
# flatten each grid to a vector
r1, r2 = xx.flatten(), yy.flatten()
r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
# horizontal stack vectors to create x1,x2 input for the model
grid = np.hstack((r1,r2))
# define the model
model = LogisticRegression()
# fit the model
model.fit(X, y)
# make predictions for the grid
yhat = model.predict(grid)
# reshape the predictions back into a grid
zz = yhat.reshape(xx.shape)
# plot the grid of x, y and z values as a surface
plt.contourf(xx, yy, zz, cmap='Paired')
# create scatter plot for samples from each class
for class_value in range(2):
    # get row indexes for samples with this class
    row_ix = np.where(y == class_value)
    # create scatter of these samples
    plt.scatter(X[row_ix, 0], X[row_ix, 1], cmap='Paired')

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

# Trial

In [ ]:
df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])

In [ ]:
for itr in range(1,11):
    a=[]
    i=0
    empty_Arr=[]
    df2.drop(df2.index, inplace=True)
    for m in consumer:
        if(i>(itr*1000)+100 and i <= (itr*1000)+100):
            a.append(m)
        if(i==(itr*1000)+100):
            break
        i=i+1
    empty_Arr=[]
    for i in range(0,1000):
        demo=(a[i][6]).decode("utf-8")
        demo=demo[2:-3]
        demo=demo.split(",")
        empty_Arr.append(demo)
    df2=pd.DataFrame(columns=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack'])
    for i in range(0,len(empty_Arr)):
        df2 = df2.append(pd.Series(empty_Arr[i], index=['timestamp','FQDN_count','subdomain_length','upper','lower','numeric','entropy','special','labels','labels_max','labels_average','longest_word','sld','len','subdomain','Target Attack']), 
            ignore_index=True)
    fh = FeatureHasher(n_features=3, input_type='string')
    #for feature timestamp
    hashed1 = fh.transform(df2[['timestamp']].astype(str).values)
    hashed1 = pd.DataFrame(hashed1.todense())
    hashed1.columns = ['timestamp_fh'+str(i) for i in hashed1.columns]
    hashed2 = fh.transform(df2[['longest_word']].astype(str).values)
    hashed2 = pd.DataFrame(hashed2.todense())
    hashed2.columns = ['longest_word_fh'+str(i) for i in hashed2.columns]
    hashed3 = fh.transform(df2[['sld']].astype(str).values)
    hashed3 = pd.DataFrame(hashed3.todense())
    hashed3.columns = ['sld_fh'+str(i) for i in hashed3.columns]
    df_1 = pd.concat([df2,hashed1,hashed2,hashed3],axis=1)
    X =df_1.drop(["timestamp", "longest_word", "sld", "Target Attack"], axis = 1)
    y = df2["Target Attack"]
    y=y.astype(float)
    #choose best features for training
    train_labels = X[['subdomain_length', 'lower', 'numeric', 'special', 'labels_max', 'len',
        'longest_word_fh0', 'longest_word_fh1', 'sld_fh0', 'sld_fh1',
        'sld_fh2']]
    train_labels = train_labels.astype(float)
    import pickle
    # save the model to disk
    filename = 'xgboost_model.sav'
    # pickle.dump(xgb_clf, open(filename, 'wb'))
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(train_labels, y)
    results.append(result)
    print(result)

In [27]:
import xgboost as xgb
from xgboost import XGBClassifier

xgb_clf = xgb.XGBClassifier(learning_rate =0.001, n_estimators=1500)
xgb_clf.fit(X_train, y_train)
xgb_clf.score(X_train,y_train) 

0.8325